In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from google.cloud import bigquery

In [2]:
#client instance
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [3]:
#SF bikeshare dataset
data_ref = client.dataset("san_francisco", project="bigquery-public-data")

data = client.get_dataset(data_ref)

In [4]:
table_ref = data_ref.table("bikeshare_trips")

table = client.get_table(table_ref)

In [5]:
df = client.list_rows(table, max_results=10).to_dataframe()

df

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,944732,2618,2015-09-24 17:22:00+00:00,Mezes,83,2015-09-24 18:06:00+00:00,Mezes,83,653,94063,Customer
1,984595,5957,2015-10-25 18:12:00+00:00,Mezes,83,2015-10-25 19:51:00+00:00,Mezes,83,52,nil,Customer
2,984596,5913,2015-10-25 18:13:00+00:00,Mezes,83,2015-10-25 19:51:00+00:00,Mezes,83,121,nil,Customer
3,1129385,6079,2016-03-18 10:33:00+00:00,Mezes,83,2016-03-18 12:14:00+00:00,Mezes,83,208,94070,Customer
4,1030383,5780,2015-12-06 10:52:00+00:00,Mezes,83,2015-12-06 12:28:00+00:00,Mezes,83,44,94064,Customer
5,1102641,801,2016-02-23 12:25:00+00:00,Mezes,83,2016-02-23 12:39:00+00:00,Mezes,83,174,93292,Customer
6,969490,255,2015-10-13 19:02:00+00:00,Mezes,83,2015-10-13 19:07:00+00:00,Mezes,83,650,94063,Subscriber
7,1129386,6032,2016-03-18 10:33:00+00:00,Mezes,83,2016-03-18 12:13:00+00:00,Mezes,83,155,94070,Customer
8,947105,1008,2015-09-26 12:57:00+00:00,Mezes,83,2015-09-26 13:13:00+00:00,Mezes,83,157,94063,Subscriber
9,1011650,60,2015-11-16 18:54:00+00:00,Mezes,83,2015-11-16 18:55:00+00:00,Mezes,83,35,94124,Subscriber


In [6]:
df.shape

(10, 11)

Loading test query over 
- using Temporary table trip_by_day using WITH... AS method
- cumulative_trip column using agg function SUM... OVER

In [9]:
query = """
            WITH trip_by_day AS
            (
                SELECT DATE(start_date) AS trip_date,
                COUNT(*) AS num_trips
                FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                WHERE EXTRACT(YEAR FROM start_date) = 2015
                GROUP BY trip_date
                )
                
            SELECT *,
                SUM(num_trips) OVER(
                    ORDER BY trip_date
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                    ) AS cumulative_trips
            FROM trip_by_day
"""

In [10]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query, job_config=safe_config)

result = query_job.to_dataframe()

result

,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284
...,...,...,...
360,2015-12-27,146,344692
361,2015-12-28,475,345167
362,2015-12-29,502,345669
363,2015-12-30,319,345988


In [ ]:
result['num_trips'].plot()